In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.impute import SimpleImputer
from catboost import CatBoostRegressor, Pool

In [3]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from catboost import CatBoostRegressor, Pool

train_data = pd.read_csv('house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('house-prices-advanced-regression-techniques/test.csv')
train_data = train_data.drop(columns=["Id"])
val_ids = test_data["Id"]
test_data = test_data.drop(columns=["Id"])

for feature in train_data.columns:
    percent = train_data[feature].isnull().sum() /  train_data.shape[0]
    if (percent > 0.7):
        train_data = train_data.drop(columns = feature)
        test_data = test_data.drop(columns = feature)

cat_features = []
iter = 0
for feature in train_data.columns[:-1]:
    if (train_data[feature].dtype == 'object'): cat_features.append(iter)
    iter +=1

X_train, X_test, y_train, y_test = train_test_split(train_data.drop(columns='SalePrice').values, np.log1p(train_data['SalePrice'].values), test_size = 0.2, random_state = 98987)
X_train = SimpleImputer(strategy='most_frequent').fit_transform(X_train)
X_test = SimpleImputer(strategy='most_frequent').fit_transform(X_test)
test_data = SimpleImputer(strategy='most_frequent').fit_transform(test_data)

In [4]:
def print_rmse(y_pred, y_test):
    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = mse ** (1/2)
    print("MSE: %.2f" % mse)
    print("RMSE: %.2f" % rmse)
    print('R**2:', metrics.r2_score(y_pred, y_test))
    rmse_model2 = np.sqrt(metrics.mean_squared_error(np.log(y_test), np.log(y_pred)))
    print('rmse log: ', rmse_model2)

In [6]:
catbr = CatBoostRegressor(cat_features=cat_features)
train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)
catbr.fit(train_pool, eval_set = test_pool, verbose = 0, plot = False)

In [7]:
y_pred = catbr.predict(X_test)
print_rmse(y_pred, y_test)

MSE: 0.02
RMSE: 0.14
R**2: 0.8510474343003671
rmse log:  0.012137004585733187


In [8]:
pred_test_data = catbr.predict(test_data)
df = pd.DataFrame({'Id':val_ids.values, 'SalePrice':pred_test_data})
df.to_csv('catboost.csv',index=False)